#### 0.0 Load libreries

In [1]:
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Synset
import nltk 
import re
import math
import scipy
from nltk.corpus import semcor
import random
import csv


print("Libreries imported successfully ✓")

Libreries imported successfully ✓


#### 0.1 definition of functions for preprocessing

In [2]:
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', str(sentence)).strip()

def get_stopwords():
    stopwords_file = open(f"./resources/utils/stop_words__frakes_baeza-yates.txt", "r")
    stopwords_list = []
    for word in stopwords_file:
        stopwords_list.append(word.replace('\n', ''))
    stopwords_file.close()

    stopwords_file = open(f"./resources/utils/stop_words_1.txt", "r")
    stopwords_list = []
    for word in stopwords_file:
        stopwords_list.append(word.replace('\n', ''))
    stopwords_file.close()

    stopwords_file = open(f"./resources/utils/stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords_file:
        stopwords_list.append(word.replace('\n', ''))
    stopwords_file.close()
    
    stopwords_list = list(set(stopwords_list))

    return stopwords_list
     
def remove_stopwords(sentence, stopwords_list):
    return [word for word in sentence.split() if word not in stopwords_list]



### Esercitazione su Word Sense Disambiguation: 

#### INPUT : 
l’input per questa esercitazione è costituito da coppie di termini contenute nel file WordSim353 (disponibile nei
formati .tsv e .csv)
- Il file contiene 353 coppie di termini utilizzati come testset in varie competizioni internazionali
- A ciascuna coppia è attribuito un valore numerico [0,10], che rappresenta la similarità fra gli elementi della coppia.

In [3]:
sim_file = f"resources/WordSim353/WordSim353.csv"

pairs = []
with open(sim_file, 'r', encoding = "utf-8") as f:
    reader = csv.reader(f)
    reader.__next__()
    for lines in reader:
        pairs.append(lines)

print("WordSim353 loaded correctly ✓")


WordSim353 loaded correctly ✓


#### Prima parte

Implementare tre misure di similarità basate su WordNet. Per ciascuna delle misure di similarità, calcolare:
- gli indici di correlazione di Spearman e
- gli indici di correlazione di Pearson fra i risultati ottenuti e quelli ‘target’ presenti nel file annotato.

Le misure da implementare sono le seguenti: 
- Wu & Palmer: 
$$cs(s_1, s_2) = \frac{2 \cdot depth(LCS)}{depth(s_1) + depth(s_2)}$$

in cui $LCS$ rappresenta il primo antenato comune (Lowest Common Subsumer) e depth è la funzione che misura la distanza fra la radice di WordNet e il sysnet $x$. 

***L’obiettivo è implementare la misura di similarità di Wu & Palmer! Non dobbiamo fare ciò viene già fatto nell’implementazione di nltk o altre librerie!***



In [4]:
def get_LCS(synset1: Synset, synset2: Synset) -> Synset or None:
    
    hypernyms1 = []
    hypernyms2 = []

    hypernyms1.append(synset1) 
    hypernyms2.append(synset2) 

    # for each synset extraction of hypernyms to add to the list -> closure of the hypernymy relation?
    for synset in hypernyms1:
        if synset_hyp := synset.hypernyms():
            hypernyms1.extend(synset_hyp)
        #print("\nPER ", str(synset), " Hypernyms are: ", synset_hyp)

    for synset in hypernyms2:
        if synset_hyp := synset.hypernyms():
            hypernyms2.extend(synset_hyp)

    if hypernyms1 and hypernyms2:
        if common_hypernyms := set(hypernyms1).intersection(set(hypernyms2)):
            common_hypernyms = [(syn, syn.max_depth() + 1) for syn in common_hypernyms] 
            common_hypernyms.sort(key=lambda x: x[1], reverse=True)
            #print("common_hypernyms max_depth: ", common_hypernyms[0])
            #print(common_hypernyms)
            #print("common_hypernyms NLTK: ", synset1.common_hypernyms(synset2))

            return common_hypernyms[0][0] #LCS: Lowest Common Subsumer
        else:
            return None
    else: 
        return None

In [5]:
def wu_palmer_similarity(syn1: Synset, syn2: Synset):
    """
    Returns the Wu-Palmer similarity between synset.
    """
    
    if isinstance(syn1, Synset) and isinstance(syn2, Synset):
        lcs_depth = 0
        lcs = None
        cs = 0

        #print("\nSYN1: " + str(syn1))
        #print("SYN2: " + str(syn2))

        if lcs := get_LCS(syn1, syn2):
            lcs_depth = lcs.max_depth() + 1
        
        #print("LCS: " + str(lcs))
        #print("LCS NLTK: " + str(syn1.lowest_common_hypernyms(syn2)))
        depth1 = syn1.min_depth()
        depth2 = syn2.min_depth()
        if depth1 > 0 and depth2 > 0:
            cs = (2 * (lcs_depth)) / ((syn1.min_depth()) + (syn2.min_depth())) 
        
        #cs = (2 * (lcs_depth)) / ((syn1.shortest_path_distance(lcs) + lcs_depth) + (syn2.shortest_path_distance(lcs) + lcs_depth)) #Così sarebbe perfetto
        
        #print("WU PALMER SIMILARITY MANUAL", cs)

        #print("WU PALMER SIMILARITY NLTK", syn1.wup_similarity(syn2, simulate_root=False))

        return cs
    else:
        raise TypeError("The input parameters must be Synset objects")

In [6]:
# TEST
#wu_palmer_similarity(wn.synsets("boat")[0], wn.synsets("computer")[0])

#wu_palmer_similarity(wn.synsets("ferry")[0], wn.synsets("boat")[0])

wu_palmer_similarity(wn.synsets("boat")[0], wn.synsets("revenge")[0])

0.125

- Shortest Path:
$$sim_{path}(s_1, s_2) = 2 \cdot depthMax - len(s_1, s_2)$$

in cui $depthMax$ è un valore fissato per una specifica versione di WordNet. 

In [7]:
#CALCOLO REALE depthMax = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())
depth_max = 20
#depth_max = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())

def path_len_between_two_synset(synset1: Synset, synset2: Synset):
    lcs = get_LCS(synset1, synset2)
    #print("LCS: ", lcs)
    #print("LCS LIBRARY: ", synset1.lowest_common_hypernyms(synset2))
    if lcs is not None:
        #print("synset1.max_depth(): ", synset1.min_depth())
        #print("synset2.max_depth(): ", synset2.min_depth())
        #print("LCS.max_depth(): ", lcs.max_depth() + 1)
        
        distance = (synset1.min_depth() - lcs.max_depth() + 1) + (synset2.min_depth() - lcs.max_depth() + 1)
        #print("DISTANCE: ", distance)
        #print("DISTANCE LIBRARY: ", (synset1.shortest_path_distance(lcs) + synset2.shortest_path_distance(lcs)))

        return distance
    return 0

def shortest_path(synset1: Synset, synset2: Synset):
    if (len_between_two_synset := path_len_between_two_synset(synset1, synset2)) >= 0:
        return 2 * depth_max - len_between_two_synset
    else: 
        return 0


In [8]:
# TEST
shortest_path_value = shortest_path(wn.synsets("dog")[0], wn.synsets("computer")[0])
print(shortest_path_value)

wn.synsets("dog")[0].path_similarity(wn.synsets("computer")[0]) # TODO: cosa rappresenta?

28


0.09090909090909091

- Leakcock & Chodorow: 
$$sim_{LC}(s_1, s_2) = - log \frac{len(s_1, s_2)}{2 \cdot depthMax}$$

In [9]:
def leakcock_chodorow(synset1, synset2):
    if (len_between_two_synset := path_len_between_two_synset(synset1, synset2)) > 0:
        fract = len_between_two_synset / (2 * depth_max) 
        return - math.log(fract)
    else: 
        return 0

In [10]:
print("MANUAL: ", leakcock_chodorow(wn.synsets("car")[0], wn.synsets("boat")[0]))

print("LIBRARY: ", wn.synsets("car")[0].lch_similarity(wn.synsets("boat")[0]))

MANUAL:  1.6094379124341003
LIBRARY:  1.55814461804655


#### Calcolo della similarità

Le funzioni precedentemente implementate richiedono come input sensi e non i termini. Quindi per calcolare la similarità fra 2 termini è necessario prendere la massima similarità fra tutti i sensi del primo termine e tutti i sensi del secondo termine. 

L'ipotesi quindi è che i due termini funzionino come contesto di disambiguazione l'uno per l'altro. 

L'equazione che formalizza questa idea è la seguente: 

$$sim(w_1, w_2) = \max_{c_1 \in s(w_1), c_2 \in s(w_2)} [sim(c_1, c_2)]$$

***NB!*** Per calcolare gli indici di correlazione non si è interessati a entrare nel merito di come sono calcolati, possiamo prendere delle funzioni prefatte e usarle.

In [11]:
def get_similarity():
    wordsimilarities = []
    wu_palmer_similarities = []
    shortpath_similarities = []
    leakcock_chodorow_similarities = []

    #Metriche
    for row in pairs:
        print("-------------------------------")
        print(row)
        term1 = row[0]
        term2 = row[1]

        synsets1 = wn.synsets(term1)
        synsets2 = wn.synsets(term2)

        print("\nsynsets1: ", synsets1)
        print("synsets2: ", synsets2)
        
        if (synsets1 != [] and synsets2 != []):

            wu_palmer_max = 0
            short_path_max = 0
            leakcock_chodorow_max = 0

            for s1 in synsets1:
                for s2 in synsets2:

                    # Metrica 1
                    #print("Wu palmer")
                    wu_palmer = wu_palmer_similarity(s1, s2)
                    # print(wu_palmer_nltk(synset1,synset2))

                    if wu_palmer > wu_palmer_max:
                        wu_palmer_max = wu_palmer

                    #Metrica 2
                    #print(shortest_path(s1, s2))
                    # print(shortest_path_nltk(synset1, synset2))
                    shortpath = shortest_path(s1, s2)

                    if shortpath > short_path_max:
                        short_path_max = shortpath

                    # Metrica 3
                    #print("Leakcock & Chodorow")
                    leacchod = leakcock_chodorow(s1, s2)
                    # print(leakcockchodrow_nltk(synset1,synset2))

                    if leacchod > leakcock_chodorow_max:
                        leakcock_chodorow_max = leacchod
            
            print("Wu palmer: ", wu_palmer_max)
            print("Shortest path: ", short_path_max)
            print("Leakcock & Chodorow: ", leakcock_chodorow_max)
            wu_palmer_similarities.append(float(wu_palmer_max))
            shortpath_similarities.append(float(short_path_max))
            leakcock_chodorow_similarities.append(float(leakcock_chodorow_max))
            
            wordsimilarities.append(float(row[2]))

    return wu_palmer_similarities, shortpath_similarities, leakcock_chodorow_similarities, wordsimilarities

In [12]:
wupalmerres, pathres, lcres, wordsimres = get_similarity()

-------------------------------
['love', 'sex', '6.77']

synsets1:  [Synset('love.n.01'), Synset('love.n.02'), Synset('beloved.n.01'), Synset('love.n.04'), Synset('love.n.05'), Synset('sexual_love.n.02'), Synset('love.v.01'), Synset('love.v.02'), Synset('love.v.03'), Synset('sleep_together.v.01')]
synsets2:  [Synset('sexual_activity.n.01'), Synset('sex.n.02'), Synset('sex.n.03'), Synset('sex.n.04'), Synset('arouse.v.07'), Synset('sex.v.02')]
Wu palmer:  1.0909090909090908
Shortest path:  40
Leakcock & Chodorow:  2.5902671654458267
-------------------------------
['tiger', 'cat', '7.35']

synsets1:  [Synset('tiger.n.01'), Synset('tiger.n.02')]
synsets2:  [Synset('cat.n.01'), Synset('guy.n.01'), Synset('cat.n.03'), Synset('kat.n.01'), Synset('cat-o'-nine-tails.n.01'), Synset('caterpillar.n.02'), Synset('big_cat.n.01'), Synset('computerized_tomography.n.01'), Synset('cat.v.01'), Synset('vomit.v.01')]
Wu palmer:  1.4
Shortest path:  40
Leakcock & Chodorow:  2.5902671654458267
-------------

In [13]:
#Indici di correlazione

print("len(wordsimres): ", len(wordsimres))
print("len(wupalmerres): ", len(wupalmerres))
print("len(pathres): ", len(pathres))
print("len(lcres): ", len(lcres))

#Pearson
print("####################################################################")
print("Pearson ")
print(scipy.stats.pearsonr(wordsimres, wupalmerres))
print(scipy.stats.pearsonr(wordsimres, pathres))
print(scipy.stats.pearsonr(wordsimres, lcres))

#Spearman
print("Spearman ")
print(scipy.stats.spearmanr(wordsimres, wupalmerres))
print(scipy.stats.spearmanr(wordsimres, pathres))
print(scipy.stats.spearmanr(wordsimres, lcres))

len(wordsimres):  352
len(wupalmerres):  352
len(pathres):  352
len(lcres):  352
####################################################################
Pearson 
(0.2600498649766201, 7.537284812818245e-07)
(0.007958368933504253, 0.8817244740405547)
(0.1623968562988756, 0.0022407905209132733)
Spearman 
SpearmanrResult(correlation=0.2833922333730243, pvalue=6.327502695974195e-08)
SpearmanrResult(correlation=-0.03194154768881925, pvalue=0.5503088486640739)
SpearmanrResult(correlation=0.1923225769763447, pvalue=0.0002840858776735863)


### Seconda parte

Implementare l’algoritmo di Lesk (NON (!=) usare implementazione esistente, e.g., in nltk…).
1. Estrarre 50 frasi dal corpus SemCor (corpus annotato con i synset di
WN) e disambiguare (almeno) un sostantivo per frase. 
Calcolare l’accuratezza del sistema implementato sulla base dei sensi annotati in
SemCor.
    - SemCor è disponibile all’URL http://web.eecs.umich.edu/~mihalcea/downloads.html
2. Randomizzare la selezione delle 50 frasi e la selezione del termine da disambiguare, 
e restituire l’accuratezza media su (per esempio) 10 esecuzioni del programma.

In [14]:
stop_words = get_stopwords()

def get_set_of_words(phrase: str):
    """
    Returns the set of words of a phrase.
    """
    phrase = remove_punctuation(phrase)
    set_of_words = remove_stopwords(phrase, stop_words)

    return set_of_words

def get_context_of_phrase(sentence: str):
    """
    Returns the context of a phrase.
    """
    sentence = remove_punctuation(sentence)
    set_of_words = remove_stopwords(sentence, stop_words)

    return set_of_words

def compute_overlap(signature, context):
    """
    Returns the number of words in common between signature and context
    """

    number_of_words_in_common = len(list(set(signature) & set(context)))

    return number_of_words_in_common

def get_signature(synset: Synset):
    """
    Returns the signature of synset, i.e. the set of words that are contained in the gloss and examples of synset.
    """

    gloss_of_synset = synset.definition() #gloss of synset
    examples_of_synset = synset.examples() #examples of synset
    initial_signature = [gloss_of_synset] + examples_of_synset

    signature = []
    for phrase in initial_signature:
        set_of_words = get_set_of_words(phrase)
        signature.extend(set_of_words)

    return list(set(signature)) # remove duplicates

In [15]:
def lesk(word: str, sentence: str):
    """
    Returns the best sense of given word used in sentence.
    """ 

    max_overlap = 0
    #context = sentence #get_context_of_phrase(sentence) #set of words in sentence
    context = get_set_of_words(sentence)
    synsets = wn.synsets(word)
    print(wn.synsets(word))


    best_sense = None #most frequent sense for word

    if synsets != []:
        best_sense = synsets[0]

    for synset in synsets:
        # TODO: CAPIRE QUALE SIGNATURE USARE -> SE USIAMO A PRIMA VENGONO TROVATI MENO SYNSET DAL LESK, ALTRIMENTI DI PIU'

        signature = get_signature(synset) #set of words in the gloss and examples of sense
        overlap = compute_overlap(signature, context)

        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = synset

    return best_sense

In [16]:
# Test
best_sense = lesk("dog", "I saw a dog in the park.")

best_sense_nltk = nltk.wsd.lesk("I saw a dog in the park.".split(), "dog")

print("best sense:")
print(best_sense) 

print("best sense nltk:", best_sense_nltk)

[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01'), Synset('chase.v.01')]
best sense:
Synset('dog.n.01')
best sense nltk: Synset('dog.n.01')


#### 1. Extraction first 50 phrases and disambiguation of (at least) one name

In [32]:
corpus = semcor.sents()

def lemma_list(sent):
    return [l.label() for l in sent if isinstance(l, nltk.tree.Tree)]

def disambiguation(sentences: list, sentences_tagged: list):
    lesk_correct = 0 
    total_names = 0 

    for sentence, sentence_tagged in zip(sentences, sentences_tagged):
        lemmas = lemma_list(sentence_tagged)

        lemmas_length = len(lemmas)
        names_disambiguated = 0 # count to stop the search of disambiguation
        
        j = 0
        while (j < lemmas_length and names_disambiguated < 3):
            string = str(lemmas[j])
            string = string.replace("Lemma('","")
            string = string.replace("')", "") # jury.n.01.jury
            synset_string = string.rsplit('.', 1)[0] # jury.n.01
            
            parts = string.split('.')   # jury.n.01.jury --> ['jury', 'n', '01', 'jury']
            
            # Lemma('jury.n.01.jury') --> "synset".paroladeltesto
            # parts[0] = jury
            # parts[1] = n (può essere 'n', 'r', 'v', ...)
            # parts[2] = 01
            # parts[3] = jury # lemma della parola del testo che ha "taggato"

            if parts[1] == 'n': #and parts[3] in sentences[i]):
                total_names += 1
                names_disambiguated += 1
                lesk_synset = lesk(parts[3],sentence) 
                print("Phrase: ", sentence)
                print("Word: "+parts[3])
                print("Lesk algorithm: " + str(lesk_synset))
                print("True Synset: " + synset_string)

                
                if lesk_synset:
                    lesk_name_syn = str(lesk_synset)
                    lesk_name_syn = lesk_name_syn.replace("Synset('", "")
                    lesk_name_syn = lesk_name_syn.replace("')","")

                    if (synset_string == lesk_name_syn): # confront synset true with lesk one 
                        lesk_correct = lesk_correct + 1

            j += 1
        lemmas.clear()
    print("Numero di volte che lesk ha trovato la giusta definizione: " + str(lesk_correct))
    print("Numero di nomi totali da disambiguare: " + str(total_names))
        

In [33]:
def first_50_phrases():
    sentences = corpus[0:50]
    sentences_tagged = semcor.tagged_sents(tag="sem")[0:50]

    return sentences, sentences_tagged

sentences, sentences_tagged = first_50_phrases()
disambiguation(sentences, sentences_tagged)

[Synset('group.n.01'), Synset('group.n.02'), Synset('group.n.03'), Synset('group.v.01'), Synset('group.v.02')]
Phrase:  ['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
Word: group
Lesk algorithm: Synset('group.n.01')
True Synset: group.n.01
[Synset('friday.n.01')]
Phrase:  ['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
Word: Friday
Lesk algorithm: Synset('friday.n.01')
True Synset: friday.n.01
[Synset('probe.n.01'), Synset('investigation.n.02')]
Phrase:  ['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``'

#### 2. Random extraction of 50 phrases from corpus

In [44]:
def generate_random_numbers_in_range(low, high, n):
    return random.sample(range(low, high), k=n)

def random_50_phrases():
    random_indexes = generate_random_numbers_in_range(0, len(corpus), 50)
    random_indexes.sort()
    sentences = []
    sentences_tagged = []
    for i in random_indexes: 
        sentences.append(corpus[i])
        sentences_tagged.append(semcor.tagged_sents(tag="sem")[i])

    return sentences, sentences_tagged

In [45]:
sentences, sentences_tagged = random_50_phrases()

disambiguation(sentences, sentences_tagged)

KeyboardInterrupt: 

In [59]:
def random_phrases():  # implementazione del secondo punto della seconda parte, con l'utilizzo di 50 frasi casuali del corpus semcor
    index_list = generate_random_numbers_in_range(0, len(corpus), 50) # genero 50 numeri casuali tra 0 e il numero di frasi nel corpus

    sentences = [corpus[i] for i in index_list]  # prendo 50 frasi casuali

    lesk_correct = 0 # numero di volte che lesk ha trovato la giusta definizione

    for i in range(0,50): 
        print("-------------")

        print("\nTAGGED")

        print(semcor.tagged_sents(tag="sem")[i], "\n\n")
        
        # recupero i lemmi dalla frase taggata con il synset per ogni parola
        lemma = lemma_list(semcor.tagged_sents(tag="sem")[i]) # :param tag: `'pos'` (part of speech), `'sem'` (semantic), or `'both'`
        print(lemma)

        print("FINE STAMPA LEMMA")

        lemma_length = len(lemma)
        noun_list = []

        j = 0
        while (j < lemma_length):
            if (str(lemma[j]) != 'None'):
                string = str(lemma[j])
                string = string.replace("Lemma('","")
                string = string.replace("')", "") # jury.n.01.jury
                synset_string = string.rsplit('.', 1)[0]

                parts = string.split('.')   # jury.n.01.jury --> ['jury', 'n', '01', 'jury']
                
                # Lemma('jury.n.01.jury') --> "synset".paroladeltesto
                # parts[0] = jury
                # parts[1] = n (può essere 'n', 'r', 'v', ...)
                # parts[2] = 01
                # parts[3] = jury # lemma della parola del testo che ha "taggato"

                if (parts[1] == 'n' and parts[3] in sentences[i]):

                    noun_list.append([parts[0], parts[1], parts[2], parts[3]]) # aggiungo alla lista dei nomi la parola, il suo synset e il lemma

            j = j+1

        print("NOUN_LIST: ", noun_list)
        print("LEN(NOUN_LIST): ", len(noun_list))

        if len(noun_list) > 0:
            selected_noun = random.choice(noun_list) # seleziono un nome a caso tra quelli presenti nella frase
            print("\nSELECTED_NOUN: ", selected_noun, "\n")

            lesk_synset = lesk(selected_noun[3], sentences[i]) # trovo il synset con lesk
            print(sentences[i])
            print("Word: "+selected_noun[3])
            print("Lesk algorithm: " + str(lesk_synset))
            print(selected_noun[0],selected_noun[1],selected_noun[2]) 

            syn = str(lesk_synset)
            if (syn != None):
                syn = syn.replace("Synset('", "")
                syn = syn.replace("')","")

                if (synset_string == syn): # confronto il synset trovato con lesk con quello "taggato" nel testo
                    lesk_correct = lesk_correct + 1

        lemma.clear()
        noun_list.clear()

    print("numero di volte che lesk ha trovato la giusta definizione: " + str(lesk_correct))

In [60]:
random_phrases()

-------------
sentences[i]:  ['Most', 'people', 'would', 'have', 'to', 'hire', 'a', 'contractor', 'to', 'build', 'this', 'shelter', '.']

TAGGED
[['The'], Tree(Lemma('group.n.01.group'), [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]), Tree(Lemma('state.v.01.say'), ['said']), Tree(Lemma('friday.n.01.Friday'), ['Friday']), ['an'], Tree(Lemma('probe.n.01.investigation'), ['investigation']), ['of'], Tree(Lemma('atlanta.n.01.Atlanta'), ['Atlanta']), ["'s"], Tree(Lemma('late.s.03.recent'), ['recent']), Tree(Lemma('primary.n.01.primary_election'), ['primary', 'election']), Tree(Lemma('produce.v.04.produce'), ['produced']), ['``'], ['no'], Tree(Lemma('evidence.n.01.evidence'), ['evidence']), ["''"], ['that'], ['any'], Tree(Lemma('abnormality.n.04.irregularity'), ['irregularities']), Tree(Lemma('happen.v.01.take_place'), ['took', 'place']), ['.']] 


[None, Lemma('group.n.01.group'), Lemma('state.v.01.say'), Lemma('friday.n.01.Friday'), None, Lemma('probe.n.01.investigation'), None, Lemma